In [1]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp

from deeplp.models.data_prep import create_weighted_graph, load_data
from deeplp.models.data_prep import prepare_data, random_unlabel, calc_masks
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.models.utils import accuracy, indices_to_vec
from deeplp.models.data_prep import select_features


/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = 'cora'
seed = 0

true_labels, features, edge_features, node_features, graph \
= load_data(data,'linqs',directed=1,confidence=0)


labeled_indices, unlabeled_indices = \
    random_unlabel(true_labels,0.99,features,
                   seed=seed,confidence=0)
num_nodes, num_classes = true_labels.shape

labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)



-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!


In [3]:
train_data, validation_data = prepare_data(labels,is_labeled,
                                       labeled_indices,None,true_labels,
                                       1,100,
                                       1)

In [4]:
for index in ['X','y','masked','true_labeled','validation_labeled','masked']:
    train_data[index] = np.vstack([train_data[index][labeled_indices],train_data[index][unlabeled_indices]])
    validation_data[index] = np.vstack([validation_data[index][labeled_indices],validation_data[index][unlabeled_indices]])

In [5]:
graph = sp.vstack([graph[labeled_indices],graph[unlabeled_indices]])

In [6]:
graph = sp.vstack([graph[labeled_indices],graph[unlabeled_indices]])
labels = np.vstack([labels[labeled_indices],labels[unlabeled_indices]])
is_labeled = np.vstack([is_labeled[labeled_indices],is_labeled[unlabeled_indices]])

In [7]:
is_labeled = np.vstack([is_labeled[labeled_indices],is_labeled[unlabeled_indices]])

In [8]:

print('Data prepared!')
sys.stdout.flush()

lp = LP()
unlabeled_pred = lp.iter_sp(labels,
                     graph.T,
                     is_labeled,
                     100,
                     unlabeled_indices)
y_pred = np.argmax(unlabeled_pred,axis=1)
y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
accuracy = np.mean(y_pred == y_true)

Data prepared!


In [9]:
accuracy

0.20121951219512196

In [22]:
# ress = []

# for data in ['cora','citeseer','pubmed']:
#     res = []
#     for seed in range(10):
#         true_labels, features, edge_features, node_features, graph \
#         = load_data(data,'linqs',directed=1,confidence=0)


#         labeled_indices, unlabeled_indices = \
#             random_unlabel(true_labels,0.99,features,
#                            seed=seed,confidence=0)
#         num_nodes, num_classes = true_labels.shape

#         labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=0)


#         from sklearn.linear_model import LogisticRegression
#         logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs',class_weight='balanced')
#         logreg.fit(features[labeled_indices], np.argmax(true_labels[labeled_indices],axis=1))
#         y_pred = logreg.predict_proba(features[unlabeled_indices])

#         acc = np.mean(np.argmax(y_pred,axis=1) == np.argmax(true_labels[unlabeled_indices],axis=1))
#         res.append(acc)
#     ress.append(res)

-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Done!
-----------cora-----------
Loading labels...
Loading features...
Load

In [14]:
shape             = [num_nodes, None, num_classes]
X            = tf.placeholder("float", shape=shape)
y            = tf.placeholder("float", shape=shape)
labeled      = tf.placeholder("float", shape=shape)
true_labeled = tf.placeholder("float", shape=shape)
validation_labeled = tf.placeholder("float", shape=shape)
masked       = tf.placeholder("float", shape=shape)

In [17]:
features = edge_features
num_features = features.shape[1]
print(num_features)
num_edges = features.shape[0]
num_nodes = graph.shape[0]
features = tf.constant(features, dtype=tf.float32)


theta = np.zeros((1,num_features))

theta = tf.Variable(theta, dtype=tf.float32)
#
# if weight_normalization == 'smooth_relu':
#     b     = tf.Variable(1, dtype=tf.float32)
#     values = tf.reduce_sum(features * theta + b,axis=1)
#     indices = tf.constant(list(zip(graph.tocoo().row,graph.tocoo().col)), dtype=tf.int64)
#     weights_unnormalized = tf.SparseTensor(indices, smooth_relu(values), [num_nodes, num_nodes])
#     weights = _normalize_weights(weights_unnormalized)

b     = tf.constant(1, dtype=tf.float32)
values = tf.reduce_sum(features * theta + b,axis=1)
indices = tf.constant(list(zip(graph.tocoo().row,graph.tocoo().col)), dtype=tf.int64)
weights_unnormalized = tf.SparseTensor(indices, values, [num_nodes, num_nodes])
weights = tf.sparse_softmax(weights_unnormalized)

39


In [23]:
features * theta + b

<tf.Tensor 'add_1:0' shape=(10138, 39) dtype=float32>

In [21]:
labeled_indices

array([  21,  100,  278,  466,  539,  707,  757,  879,  979, 1383, 1390,
       1706, 1783, 1833, 1866, 1877, 1946, 2014, 2052, 2126, 2141, 2169,
       2228, 2229, 2369])

In [13]:
np.where(train_data['labeled']==1)

(array([  21,   21,   21, ..., 2369, 2369, 2369]),
 array([ 0,  0,  0, ..., 24, 24, 24]),
 array([0, 1, 2, ..., 4, 5, 6]))

In [11]:
trained_model = DeepLP_Edge(0,
                    'edge',
                    'test',
                    edge_features, # [:,6:] for edge features only
                    graph,
                    labels.shape[1],
                    unlabeled_indices,
                    loss_class_mass=1,
                    confidence=0,
                    regularize=0,
                    num_iter=100,
                    clamp=1,
                    lr=0.001,
                    seed=0)

39
Instructions for updating:
keep_dims is deprecated, use keepdims instead
start
Tensor("Placeholder:0", shape=(2485, ?, 7), dtype=float32)
middle
Tensor("while/truediv_2:0", shape=(2485, ?, 7), dtype=float32) Tensor("Placeholder_2:0", shape=(2485, ?, 7), dtype=float32) Tensor("Placeholder:0", shape=(2485, ?, 7), dtype=float32)


In [12]:
trained_model.train(train_data,validation_data,10)

[[1.         1.         1.         ... 1.         0.9999999  1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         0.99999994]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [0.99999994 1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 0.99999994 1.         1.        ]] (2485, 25)
==  epoch: -1 labeled loss: 0.24698071 regularize_val 0 regularize_param 0 objective 0.24698071 unlabeled loss: 0.23978232 accuracy: 0.63318974 entropy: 1.8455542 kl_uniform: 14.313061 kl_class_prior: 1.984911 true unlabeled loss: 0.23781744 true accuracy: 0.6406504 validation_accuracy: 0.6406504 clamp_a: 1.0 clamp_c: 1.0 signaltonoise: inf entropy_sd: 0.0
===  lambda2: 0.5773267 weights: [0.2  0.2  0.2  0.2  0.2  0.25 0.25 0.25 0.25 1.  ] weights_unnormalized: [39. 39. 39. 39. 39. 39. 39. 39. 39. 39.]


/Users/kojin/projects/deep_label_propagation/deeplp/models/deeplp.py:551: RuntimeWarning: divide by zero encountered in float_scalars
  signaltonoise = entropy / entropy_sd
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kojin/anaconda/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


== theta_edge: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
== b: 1.0
== theta: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--- 10.814852952957153 seconds ---
[[1.         1.         1.         ... 1.         1.         0.99999994]
 [1.         1.         1.         ... 1.         0.99999994 1.        ]
 [0.99999994 1.         1.0000001  ... 1.         0.9999999  1.        ]
 ...
 [1.         0.99999994 0.99999994 ... 0.99999994 1.         1.        ]
 [1.         0.99999994 0.99999994 ... 1.         1.         1.        ]
 [1.0000001  0.99999994 1.         ... 1.0000001  0.99999994 0.99999994]] (2485, 25)
==  epoch: 0 labeled loss: 0.24672276 regularize_val 0 regularize_param 0 objective 0.24672276 unlabeled loss: 0.23953365 accuracy: 0.6329297 entropy: 1.8448074 kl_uniform: 14.31766 kl_class_prior: 1.9851862 true unlabeled loss: 0.23756543 true accuracy: 0.6406504 validation_accuracy: 0.6406504 clamp_a: 1.0 clamp_c: 1.

==  epoch: 4 labeled loss: 0.24568042 regularize_val 0 regularize_param 0 objective 0.24568042 unlabeled loss: 0.23852731 accuracy: 0.6306867 entropy: 1.8417308 kl_uniform: 14.336646 kl_class_prior: 1.9863347 true unlabeled loss: 0.23654562 true accuracy: 0.63821137 validation_accuracy: 0.63821137 clamp_a: 1.0 clamp_c: 1.0005002 signaltonoise: inf entropy_sd: 0.0
===  lambda2: 0.57687384 weights: [0.19870028 0.20032747 0.20044062 0.19978184 0.20074975 0.24913678
 0.25003555 0.25119233 0.24963528 1.        ] weights_unnormalized: [38.98245  38.990604 38.99117  38.987877 38.99271  38.987816 38.991417
 38.996033 38.989815 38.9911  ]
== theta_edge: [[-5.0004288e-03 -4.9981307e-03  4.5984760e-03  1.5624993e-06
   9.9786048e-06  2.2673721e-06  5.9567501e-06 -2.9937230e-06
  -3.4130489e-06 -3.5513717e-06 -1.9650199e-06  6.2538061e-06
  -5.0040050e-03 -4.9528893e-07  1.4899384e-05  1.3667118e-05
   9.4739753e-06 -1.5339562e-06 -2.0539053e-06  9.1538313e-06
   4.2498177e-06 -5.0008092e-03 -4.99

== theta_edge: [[-9.0043247e-03 -8.9937709e-03  8.4481789e-03  4.8561751e-06
   9.1072925e-06  4.0418001e-07  1.8172457e-05 -6.7957571e-06
  -6.7755336e-06  6.6388992e-08 -4.2955285e-06  1.8080427e-05
  -9.0231691e-03 -2.0718146e-06  2.8235780e-05  2.7943661e-05
   2.2298633e-05 -3.8688136e-06 -2.7388860e-06  1.6651968e-05
   1.6392254e-05 -9.0059377e-03 -9.0106903e-03 -6.2592051e-07
  -2.9263792e-06 -9.0017095e-03 -9.0066642e-03 -9.0092244e-03
   8.9992648e-03  8.9930445e-03 -9.0217255e-03  1.9417012e-06
  -7.9038118e-06  1.1738083e-05 -7.1304976e-06 -9.7099610e-06
   9.0114260e-03  8.9958627e-03  8.9997966e-03]]
== b: 1.0
== theta: [-9.0043247e-03 -8.9937709e-03  8.4481789e-03  4.8561751e-06
  9.1072925e-06  4.0418001e-07  1.8172457e-05 -6.7957571e-06
 -6.7755336e-06  6.6388992e-08]
--- 10.416388273239136 seconds ---
[[1.0000001  1.         1.         ... 1.         1.         0.9999999 ]
 [0.99999994 0.9999999  0.99999994 ... 1.         1.         1.        ]
 [1.         1.        